In [100]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

In [317]:
df=pd.read_csv('D:/Python/Github_ML/Machine-Learning/data/_preprocessed.csv', index_col=0)


In [318]:
idx=df.index[df['product-type']==0]

In [319]:
idx=idx[:3200]


In [320]:
df=df.drop(labels=idx, axis=0)

In [321]:
df.shape

(8185, 22)

In [322]:
df=df.reindex(columns=[
 'user-age',
 'user-income',
 'user-savings',
 'user-properties',                   
 'user-dependents',  
 'user-pension',                   
 'product-term',   
 'year',
 'month',
 'score',
    
 'user-gender',
 'user-nationality',
 'user-knowledge',
 'user-loyalty',
 'user-loan',
 'user-riskAversion',
 'user-marital',
 'product-yield',  
 'product-risk',
    
 'product-type'
 
])

In [323]:
X = df.iloc[:, :-1].values
y = df["product-type"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=44)

In [324]:
v=np.split(X_train, [10], axis=1)
w=np.split(X_test, [10], axis=1)

In [325]:
scaler = StandardScaler()
scaler.fit(v[0])


X_train=np.hstack((scaler.transform(v[0]), v[1]))

X_test=np.hstack((scaler.transform(w[0]), w[1]))

In [326]:
# Оценка
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def error_estimation(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred) 
    print("MSE: ",mse)
    
    print("RMSE: {}".format(np.sqrt(mse)))
    
    mae = mean_absolute_error(y_test, y_pred)
    print("MAE: ",mae)

    print("R2: ", r2_score(y_test, y_pred))

## Bagging

In [327]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [328]:
clf = BaggingClassifier()
clf.fit(X_train, y_train)

BaggingClassifier()

In [329]:
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[313 195  90]
 [261 189 119]
 [210 165  95]]
              precision    recall  f1-score   support

           0       0.40      0.52      0.45       598
           1       0.34      0.33      0.34       569
           2       0.31      0.20      0.25       470

    accuracy                           0.36      1637
   macro avg       0.35      0.35      0.35      1637
weighted avg       0.36      0.36      0.35      1637



## Boosting

In [330]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [331]:
bcl = GradientBoostingClassifier()
bcl.fit(X_train, y_train)

GradientBoostingClassifier()

In [332]:
y1_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y1_pred))
print(classification_report(y_test, y1_pred))

[[313 195  90]
 [261 189 119]
 [210 165  95]]
              precision    recall  f1-score   support

           0       0.40      0.52      0.45       598
           1       0.34      0.33      0.34       569
           2       0.31      0.20      0.25       470

    accuracy                           0.36      1637
   macro avg       0.35      0.35      0.35      1637
weighted avg       0.36      0.36      0.35      1637



## Stacking

In [333]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [334]:
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svr', make_pipeline(StandardScaler(), LinearSVC(random_state=42, max_iter=100)))]

clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(X_train, y_train)

D:\Annexes\forConda\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
D:\Annexes\forConda\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
D:\Annexes\forConda\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
D:\Annexes\forConda\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
D:\Annexes\forConda\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblin

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(n_estimators=10,
                                                       random_state=42)),
                               ('svr',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('linearsvc',
                                                 LinearSVC(max_iter=100,
                                                           random_state=42))]))],
                   final_estimator=LogisticRegression())

In [335]:
y2_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y2_pred))
print(classification_report(y_test, y2_pred));

[[339 213  46]
 [248 268  53]
 [227 197  46]]
              precision    recall  f1-score   support

           0       0.42      0.57      0.48       598
           1       0.40      0.47      0.43       569
           2       0.32      0.10      0.15       470

    accuracy                           0.40      1637
   macro avg       0.38      0.38      0.35      1637
weighted avg       0.38      0.40      0.37      1637



# Task 2

In [336]:
from catboost import * 

In [337]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    task_type = "CPU",
    iterations = 45,
    learning_rate = 0.1,
    use_best_model = True,
    loss_function ='MultiClass'
)

model.fit(X_train, y_train,
         eval_set = (X_test, y_test),
         verbose = 30,
         plot = True
)

#y_pred = model.predict(X_test)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.0944446	test: 1.0970837	best: 1.0970837 (0)	total: 8.25ms	remaining: 363ms
30:	learn: 1.0425042	test: 1.0834979	best: 1.0834979 (30)	total: 223ms	remaining: 101ms
44:	learn: 1.0267805	test: 1.0818893	best: 1.0818893 (44)	total: 318ms	remaining: 0us

bestTest = 1.081889259
bestIteration = 44



In [338]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, model.predict(X_test)))

[[313 195  90]
 [261 189 119]
 [210 165  95]]
              precision    recall  f1-score   support

           0       0.41      0.61      0.49       598
           1       0.39      0.40      0.39       569
           2       0.32      0.11      0.16       470

    accuracy                           0.39      1637
   macro avg       0.37      0.37      0.35      1637
weighted avg       0.37      0.39      0.36      1637

